In [1]:
import scipy
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

for m in [scipy, np, pd, matplotlib]:
    print(m.__name__, m.__version__)

from scipy.signal import savgol_filter
from scipy.signal import argrelextrema
from scipy.signal import find_peaks

from funcs.factor_utils import FactorUtil

scipy 1.15.2
numpy 1.26.4
pandas 2.2.3
matplotlib 3.10.1


In [2]:
bar_df_1 = FactorUtil.get_bar_df('FG2409_MIN_2', '20240813', '00:00:00', 300)
bar_df_1.head()

,O,H,L,C,V,AP,AV,BP,BV
ts,,,,,,,,,
2024-08-13 09:00:00,1277.0,1278.0,1276.0,1276.0,691,1278.000,1082,1276.000,899
2024-08-13 09:02:00,1277.0,1282.0,1276.0,1278.0,7240,1278.636,65355,1277.547,48771
2024-08-13 09:04:00,1277.0,1282.0,1277.0,1280.0,5372,1280.486,75326,1279.519,80045
2024-08-13 09:06:00,1280.0,1282.0,1280.0,1282.0,3819,1281.535,130820,1280.587,136308
2024-08-13 09:08:00,1281.0,1285.0,1281.0,1284.0,8303,1282.603,179700,1281.889,149747


In [3]:
bar_df_3 = FactorUtil.get_window_bar_df('FG2409_MIN_2', '20240813')
bar_df_3.columns

20240812 22:42:00


Index(['O_9', 'H_9', 'L_9', 'C_9', 'V_9', 'AP_9', 'AV_9', 'BP_9', 'BV_9',
       'O_8', 'H_8', 'L_8', 'C_8', 'V_8', 'AP_8', 'AV_8', 'BP_8', 'BV_8',
       'O_7', 'H_7', 'L_7', 'C_7', 'V_7', 'AP_7', 'AV_7', 'BP_7', 'BV_7',
       'O_6', 'H_6', 'L_6', 'C_6', 'V_6', 'AP_6', 'AV_6', 'BP_6', 'BV_6',
       'O_5', 'H_5', 'L_5', 'C_5', 'V_5', 'AP_5', 'AV_5', 'BP_5', 'BV_5',
       'O_4', 'H_4', 'L_4', 'C_4', 'V_4', 'AP_4', 'AV_4', 'BP_4', 'BV_4',
       'O_3', 'H_3', 'L_3', 'C_3', 'V_3', 'AP_3', 'AV_3', 'BP_3', 'BV_3',
       'O_2', 'H_2', 'L_2', 'C_2', 'V_2', 'AP_2', 'AV_2', 'BP_2', 'BV_2',
       'O_1', 'H_1', 'L_1', 'C_1', 'V_1', 'AP_1', 'AV_1', 'BP_1', 'BV_1', 'O',
       'H', 'L', 'C', 'V', 'AP', 'AV', 'BP', 'BV'],
      dtype='object')

In [4]:
# 收益率 R
bar_df_3['R'] = (bar_df_3['C'] - bar_df_3['C_5']) / bar_df_3['C_5']
# 收益值 R1
bar_df_3['R1'] = (bar_df_3['C'] - bar_df_3['C_5'])

In [6]:
bar_df_3[['C_9', 'C_7', 'C_5', 'C_3', 'C_1', 'C', 'R', 'R1']].head()

,C_9,C_7,C_5,C_3,C_1,C,R,R1
ts,,,,,,,,
2024-08-13 09:00:00,1272.0,1269.0,1273.0,1276.0,1277.0,1276.0,0.002357,3.0
2024-08-13 09:02:00,1271.0,1272.0,1275.0,1275.0,1276.0,1278.0,0.002353,3.0
2024-08-13 09:04:00,1269.0,1273.0,1276.0,1277.0,1278.0,1280.0,0.003135,4.0
2024-08-13 09:06:00,1272.0,1275.0,1275.0,1276.0,1280.0,1282.0,0.005490,7.0
2024-08-13 09:08:00,1273.0,1276.0,1277.0,1278.0,1282.0,1284.0,0.005482,7.0


In [7]:
min_val, max_val = bar_df_3['R'].min(), bar_df_3['R'].max()
min_val, max_val

(-0.012924071082390954, 0.012976480129764802)

In [8]:
min_val_1, max_val_1 = bar_df_3['R1'].min(), bar_df_3['R1'].max()
min_val_1, max_val_1

(-16.0, 16.0)

In [9]:
bar_df_3['R1'].describe()

count    1584.000000
mean       -0.383838
std         3.628050
min       -16.000000
25%        -2.000000
50%         0.000000
75%         2.000000
max        16.000000
Name: R1, dtype: float64

In [10]:
bar_df_3['R'].describe()

count    1584.000000
mean       -0.000312
std         0.002951
min        -0.012924
25%        -0.001648
50%         0.000000
75%         0.001609
max         0.012976
Name: R, dtype: float64

In [11]:
# 根据值生成桶边界
bins = pd.interval_range(start=min_val, end=max_val, periods=10)
bins

IntervalIndex([ (-0.012924071082390954, -0.010334015961175377],
               (-0.010334015961175377, -0.0077439608399598025],
               (-0.0077439608399598025, -0.005153905718744227],
                (-0.005153905718744227, -0.002563850597528651],
                (-0.002563850597528651, 2.620452368692558e-05],
                (2.620452368692558e-05, 0.0026162596449025004],
                 (0.0026162596449025004, 0.005206314766118075],
                  (0.005206314766118075, 0.007796369887333652],
                  (0.007796369887333652, 0.010386425008549229],
                  (0.010386425008549229, 0.012976480129764802]],
              dtype='interval[float64, right]')

In [12]:
bins_1 = pd.interval_range(start=min_val_1, end=max_val_1, periods=10)
bins_1

IntervalIndex([                           (-16.0, -12.8],
                                           (-12.8, -9.6],
                              (-9.6, -6.399999999999999],
               (-6.399999999999999, -3.1999999999999993],
                              (-3.1999999999999993, 0.0],
                                (0.0, 3.200000000000003],
                  (3.200000000000003, 6.400000000000002],
                  (6.400000000000002, 9.600000000000001],
                               (9.600000000000001, 12.8],
                                            (12.8, 16.0]],
              dtype='interval[float64, right]')

In [13]:
# 分桶
# bar_df_3['K'] = pd.cut(bar_df_3['R'], bins=bins, labels=range(10), include_lowest=True).cat.codes
bar_df_3['K'] = pd.cut(bar_df_3['R'], bins=bins, labels=range(10), include_lowest=True)
# 值分布
bar_df_3['K'].value_counts().sort_index()

K
(-0.012924071082390954, -0.010334015961175377]       3
(-0.010334015961175377, -0.0077439608399598025]     23
(-0.0077439608399598025, -0.005153905718744227]     55
(-0.005153905718744227, -0.002563850597528651]     175
(-0.002563850597528651, 2.620452368692558e-05]     705
(2.620452368692558e-05, 0.0026162596449025004]     452
(0.0026162596449025004, 0.005206314766118075]      126
(0.005206314766118075, 0.007796369887333652]        35
(0.007796369887333652, 0.010386425008549229]         3
(0.010386425008549229, 0.012976480129764802]         6
Name: count, dtype: int64

In [14]:
# 分桶
# bar_df_3['K'] = pd.cut(bar_df_3['R'], bins=bins_1, labels=range(10), include_lowest=True).cat.codes
bar_df_3['K1'] = pd.cut(bar_df_3['R1'], bins=bins_1, labels=range(10), include_lowest=True)
# 值分布
bar_df_3['K1'].value_counts().sort_index()

K1
(-16.0, -12.8]                                 3
(-12.8, -9.6]                                 22
(-9.6, -6.399999999999999]                    56
(-6.399999999999999, -3.1999999999999993]    168
(-3.1999999999999993, 0.0]                   712
(0.0, 3.200000000000003]                     452
(3.200000000000003, 6.400000000000002]       126
(6.400000000000002, 9.600000000000001]        34
(9.600000000000001, 12.8]                      4
(12.8, 16.0]                                   6
Name: count, dtype: int64